In [31]:
import pandas as pd
pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\Users\Note reserva\Desktop\Curso Engenharia Dados\Arquivos\01. Postgree\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')

colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )
df.head(3)

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C


In [32]:
df.dtypes

Numero_da_Ocorrencia            int64
Classificacao_da_Ocorrencia    object
Data_da_Ocorrencia             object
Municipio                      object
UF                             object
Regiao                         object
Nome_do_Fabricante             object
Modelo                         object
dtype: object

In [33]:
df['Data_da_Ocorrencia'] = pd.to_datetime(df['Data_da_Ocorrencia'])

In [34]:
from datetime import datetime

ano_atual = datetime.now().year
print(ano_atual)


2025


In [35]:
df = df[df['Data_da_Ocorrencia'].dt.year == ano_atual]
df.head(10)

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo


In [36]:

from sqlalchemy import create_engine ,Integer, String, Date,VARCHAR,text

dbname   = 'python'
user     = 'postgres'
password = '1022'
host     = 'localhost'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy' 

# Deletar registros com base no ano atual
cursor=engine.connect() 
delete = text(f'delete from public.anac_sqlalchemy where extract(year from "Data_da_Ocorrencia")= {ano_atual}')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrame para o banco de dados
df.to_sql(nome_tabela, engine, index=False, if_exists='append',
                     dtype={ 
                           'Numero_da_Ocorrencia' :   Integer ,
                           'Classificacao_da_Ocorrencia': VARCHAR(50),
                           'Data_da_Ocorrencia':Date  
                           })




engine.dispose()
cursor.close()